In [36]:
from Data_Handler.DataReader import DataReader
import numpy as np
import pandas as pd
import os
from collections import Counter

In [37]:
dataReader = DataReader()

df = pd.read_csv(filepath_or_buffer=os.getenv('INTERACTIONS_AND_IMPRESSIONS_PATH'),
                                                   sep=',',
                                                   names=[
                                                       'UserID', 'ItemID', 'Impressions', 'Data'],
                                                   header=0,
                                                   dtype={'UserID': np.int32, 'ItemID': np.int32, 'Impressions': np.object0, 'Data': np.int32})

In [38]:
df = df.drop(['UserID'], axis=1)
df = df.drop(['Data'], axis=1)
df = df.dropna()
# add a comma at the end of each impression string in order to concat properly then
df['Impressions'] = df['Impressions'].apply(lambda x: str(x)+',')
df = df.groupby(['ItemID'], as_index=False)
# to concat impressions of each item
impressions_per_item = df['Impressions'].apply(sum)
impressions_per_item

,ItemID,Impressions
0,0,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19..."
1,1,"1,2,3,6607,235,7,149,108,8,9,10,11,12,13,15,16..."
2,2,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18..."
3,3,"1,2,2553,3,6607,235,5,6,7,149,108,8,10,11,12,1..."
4,4,"2303,147,475,278,750,1938,2304,1011,4,481,1017..."
...,...,...
21122,21639,"2033,1195,21639,2036,459,2037,1197,2038,1198,2..."
21123,21640,"53,21640,56,57,58,59,60,61,62,596,"
21124,21641,"319,63,64,65,66,67,68,713,69,71,73,74,75,21641..."
21125,21642,"1185,1657,1141,1142,1143,671,1186,1645,1187,11..."


In [39]:
f = open("super_powerful_URM.csv", "w+")
f.write("UserID,ItemID,Data\n")

#init_data = {'UserID': [], 'ItemID': [], 'Data': []}
#result = pd.DataFrame(init_data)

item_ids = impressions_per_item['ItemID'].unique()


for item_id in item_ids:
    impressions = impressions_per_item[impressions_per_item['ItemID']
                                    == item_id]['Impressions']
    impressions = impressions.iloc[0].split(",")
    # remove last element which is a '' due to last ','
    impressions = np.delete(impressions, -1) # array of strings
    impressions= [int(x) for x in impressions] # convert into array of integers

    for impression_item_id in impressions:
        if(impression_item_id!=item_id):
            #data = {'UserID': [impression_item_id+50000], 'ItemId': [item_id], 'Data': [1]}
            #to_append = pd.DataFrame(data)
            #result = pd.concat([result, to_append], ignore_index=True).sort_values(['UserID', 'ItemID'])
            #result = result.drop_duplicates(keep='first')
            f.write(f"{impression_item_id+50000},{item_id},1\n")
        

In [49]:
from Data_Handler.DataReader import DataReader
import numpy as np
import pandas as pd
import os
from collections import Counter
"""
    Returns super_powerful_urm that stack URM, ICM.T, and impressions.T
"""
# Vertical stack so ItemIDs cardinality must coincide.

urm=DataReader().load_augmented_binary_urm_df()
f=DataReader().load_icm_df()
swap_list = ["feature_id", "item_id", "data"]
f = f.reindex(columns=swap_list)
f = f.rename({'feature_id': 'UserID', 'item_id': 'ItemID', 'data': 'Data'}, axis=1)

urm['Data'] = 0.74 * urm['Data']
# f times (1-aplha)
f['Data'] = 0.16 * f['Data']
# Change UserIDs of f matrix in order to make recommender work
f['UserID'] = 41634 + f['UserID']

impressionURM = pd.read_csv(filepath_or_buffer="C:/rec-sys-challenge/Dataset/impressions_URM.csv",
                                            sep=',',
                                            names=[
                                                'UserID', 'ItemID', 'Data'],
                                            header=0,
                                            dtype={'UserID': np.int32, 'ItemID': np.int32, 'Data': np.int32})
# removing duplicated (user_id,item_id) pairs
impressionURM = impressionURM.drop_duplicates(keep='first')

impressionURM['Data']= 0.1 * impressionURM['Data']

powerful_urm = pd.concat([urm, f], ignore_index=True).sort_values(['UserID', 'ItemID'])
super_powerful_urm = pd.concat([powerful_urm,impressionURM], ignore_index=True).sort_values(['UserID', 'ItemID'])

super_powerful_urm

,UserID,ItemID,Data
0,0,11,0.825
1,0,21,0.825
2,0,22,0.825
3,0,24,0.825
4,0,44,0.825
...,...,...,...
1789515,71455,3274,0.150
1789517,71456,3274,0.150
1882206,71487,6258,0.150
1791363,71585,3322,0.150
